### Configure the environment

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../src')

### Compare basic classifiers

In [3]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from helper import load_data, score_model

pd.set_option("display.max_columns", None)


def create_model(classifier) -> Pipeline:
    tfidf = FeatureUnion(
        [
            ("word", TfidfVectorizer()),
            ("char", TfidfVectorizer(analyzer="char")),
        ]
    )

    classifier = CalibratedClassifierCV(
        classifier,
        cv=5,
        method="isotonic",
    )

    pipeline = Pipeline([("tfidf", tfidf), ("cls", classifier)])
    return pipeline


def compare():
    classifiers = [
        LinearSVC(dual="auto"),
        LogisticRegression(),
        XGBClassifier(),
        LGBMClassifier(verbose=-1),     
        RandomForestClassifier(),
        MultinomialNB(),
    ]

    X_train, y_train = load_data("../data/train.parquet")

    metrics_table = []

    for classifier in classifiers:

        claasifier_name = classifier.__class__.__name__
        print(f"Experiment '{claasifier_name}' in progress...")

        model = create_model(classifier)
        scores = score_model(model, X_train, y_train)

        metrics = {"classifier": claasifier_name}
        for name, values in scores.items():
            value = values.mean()
            metrics[name] = value

        metrics_table.append(metrics)

    print('Training done.')  

    df_metrics = pd.DataFrame.from_records(metrics_table)
    df_metrics = df_metrics.set_index("classifier")

    return df_metrics


df_metrics = compare()
df_metrics

Experiment 'LinearSVC' in progress...
Experiment 'LogisticRegression' in progress...
Experiment 'XGBClassifier' in progress...
Experiment 'LGBMClassifier' in progress...
Experiment 'RandomForestClassifier' in progress...
Experiment 'MultinomialNB' in progress...
Training done.


,fit_time,score_time,test_recall,test_precision,test_f1,test_accuracy,test_roc_auc
classifier,,,,,,,
LinearSVC,0.180036,0.052878,0.898928,0.932243,0.915157,0.916690,0.965994
LogisticRegression,0.523422,0.055417,0.879083,0.890518,0.884721,0.885480,0.950208
XGBClassifier,5.616612,0.113365,0.883261,0.898034,0.890580,0.891487,0.956427
LGBMClassifier,55.034700,0.102106,0.887441,0.900387,0.893861,0.894621,0.959218
RandomForestClassifier,14.628869,0.350210,0.843830,0.905621,0.873547,0.877908,0.953807
MultinomialNB,0.119692,0.056103,0.910160,0.891352,0.900520,0.899453,0.963431


### Add a little color to make it easier to read

In [4]:
df_metrics = df_metrics.sort_values("test_f1", ascending=False)
df_style = df_metrics.style

df_style.highlight_max(
    subset=df_metrics.columns[2:],
    props="background-color:lightblue;color:black"
)

df_style.bar(df_metrics.columns[:2], color='LightSalmon', width=50, height=50)
df_style

,fit_time,score_time,test_recall,test_precision,test_f1,test_accuracy,test_roc_auc
classifier,,,,,,,
LinearSVC,0.180036,0.052878,0.898928,0.932243,0.915157,0.916690,0.965994
MultinomialNB,0.119692,0.056103,0.910160,0.891352,0.900520,0.899453,0.963431
LGBMClassifier,55.034700,0.102106,0.887441,0.900387,0.893861,0.894621,0.959218
XGBClassifier,5.616612,0.113365,0.883261,0.898034,0.890580,0.891487,0.956427
LogisticRegression,0.523422,0.055417,0.879083,0.890518,0.884721,0.885480,0.950208
RandomForestClassifier,14.628869,0.350210,0.843830,0.905621,0.873547,0.877908,0.953807
